# Fully Adaptive Bayesian Algorithm for Data Analysis
# FABADA

In this notebook we show the different results obtain in the article *Sanchez-Alarcon & Ascasibar 2022*. 

### 2 DIMENSION GRAY IMAGES

In [128]:
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from PIL import Image
from ipywidgets import widgets, interactive, HBox
import pandas as pd
import os
#%matplotlib widget

In [157]:
def PSNR(data,real,n=255):
    data = np.array(data)
    real = np.array(real)
    MSE = np.sum((data-real)**2)/(data.size)
    return 10*np.log10((n)**2 / MSE)

image_names = widgets.Dropdown(
    options=['bubble','cluster','crab','eagle','galaxies','ghost','saturn','stars'] ,
    value='bubble',
    description='{:15}'.format('Image Name:'),
)

method_names = widgets.Dropdown(
    options=[('FABADA','fabada'),('BM3D','bm3d'),('Wiener Filter','wiener'),('Gaussian Filter','fft'),
    ('Savitzky Golay Filter','sgf'),('Median Filter','median')],
    value='fabada',
    description='{:15}'.format('Method:'),
)

noise_level = widgets.Dropdown(
    options=[5,10,15,25,40,60,95,125,200,255,400,510,765,1024],
    value=5,
    description='{:15}'.format('Noise level:'),
)

range_slide = widgets.FloatRangeSlider(
    value=[0.001, 255],
    min=0.001,
    max=255.0,
    step=0.1,
    description='Range (%):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

scale = widgets.ToggleButtons(
    options=['Linear', 'Log'],
    description='Scale:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#   tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)
def plotit(name, method,sigma,range,scale):
    
    original_name = os.path.join("Results",'Results_images',name,name+".png")
    noisy_name = os.path.join('Results','Results_images',name,name+'_'+'{:04d}'.format(sigma)+'_data.png')
    estimation_name_1 = os.path.join('Results','Results_images',name,name+'_'+'{:04d}'.format(sigma)+'_'+method+".png")
    methods_dict = {'fabada':'FABADA','bm3d':'BM3D','wiener':'Wiener Filter','fft':'Gaussian Filter',
    'sgf':'Savitzky Golay Filter','median':'Median Filter'} 

    original_image = Image.open(original_name).convert('L')
    noisy_image = Image.open(noisy_name).convert('L')
    estimation_1 = Image.open(estimation_name_1).convert('L')
    fig,ax = plt.subplots(1,3,figsize = (18,7),sharex=True,sharey=True)

    if scale == 'Linear':
        arg = {'vmin':range[0],'vmax':range[1]}
    else:
        arg = {'norm':LogNorm(vmin=range[0],vmax=range[1])}

    ax[0].set_title(name+" - Original",fontsize=16)
    im = ax[0].imshow(original_image,cmap="gray",**arg)

    ax[1].set_title('Noisy - {:2.2f} dB'.format(PSNR(original_image,noisy_image)),fontsize=16)
    ax[1].imshow(noisy_image,cmap="gray",**arg)

    ax[2].set_title('{} - {:2.2f} dB'.format(methods_dict[method],PSNR(original_image,estimation_1)),fontsize=16)
    ax[2].imshow(estimation_1,cmap="gray",**arg)

    ax[0].axis('off');ax[1].axis('off');ax[2].axis('off')
    plt.tight_layout()
    pos_bar = [0.1, 0.001, 0.8, 0.04]
    cax = fig.add_axes(pos_bar)
    fig.colorbar(im, cax=cax,orientation="horizontal", pad=0.2)
    cax.xaxis.set_ticks_position("bottom")

    plt.show()

    


#interactive(plotit, name=image_names, method=method_names, sigma=noise_level,range=range_slide,scale=scale)
out = interactive(plotit, name=image_names, method=method_names, sigma=noise_level,range=range_slide,scale=scale)
display(HBox(out.children[:-3]))#Show all controls
display(HBox(out.children[-3:-1]))
display(out.children[-1])#Show the output

Output()

### 1 DIMENSION SPECTRA

In [155]:
def PSNR(data,real,n=255):
    data = np.array(data)
    real = np.array(real)
    MSE = np.sum((data-real)**2)/(data.size)
    return 10*np.log10((n)**2 / MSE)

image_names = widgets.Dropdown(
    options=['Arp256','Kurucz','SN132D'] ,
    value='Arp256',
    description='{:15}'.format('Image Name:'),
)

method_names = widgets.Dropdown(
    options=[('FABADA','FABADA'),('LOESS','LOESS'),('Wiener Filter','Wiener'),('Gaussian Filter','Fourier Transform'),
    ('Savitzky Golay Filter','Savitzky–Golay'),('Median Filter','Median')],
    value='FABADA',
    description='{:15}'.format('Method:'),
)

noise_level = widgets.Dropdown(
    options=np.arange(5,100,5),
    value=5,
    description='{:15}'.format('Noise level:'),
)

range_slide = widgets.FloatRangeSlider(
    value=[0.001, 255],
    min=0.001,
    max=255.0,
    step=0.1,
    description='Range (%):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

scale = widgets.ToggleButtons(
    options=['Linear', 'Log'],
    description='Scale:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#   tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)
def plotit(name, method,sigma,range,scale):
    
    
    frame_path = os.path.join('Results','Results_spectra',name,name+'_'+'{:03d}.csv'.format(sigma))
    methods_dict = {'FABADA':'FABADA','LOESS':'LOESS','Wiener':'Wiener Filter','Fourier Transform':'Gaussian Filter',
    'Savitzky–Golay':'Savitzky Golay Filter','Median':'Median Filter'} 

    df = pd.read_csv(frame_path,delimiter=",")
    estimation = df[method]
    

    fig,ax = plt.subplots(1,1,figsize = (18,7),sharex=True,sharey=True)

    if scale == 'Linear':
        arg = {'vmin':range[0],'vmax':range[1]}
    else:
        arg = {'norm':LogNorm(vmin=range[0],vmax=range[1])}

    
    
    
    #ax.plot(noisy_image,cmap="gray",**arg)

    ax.plot(estimation,"r-",label='{} - {:2.2f} dB'.format(methods_dict[method],15.34))
    ax.set_xlabel('Arbitrary Units',fontsize=15)
    ax.set_ylabel('Arbitrary Units',fontsize=15)
    plt.tight_layout()
    plt.legend(fancybox=True,fontsize=15,ncol=3)

    plt.show()

    


#interactive(plotit, name=image_names, method=method_names, sigma=noise_level,range=range_slide,scale=scale)
out = interactive(plotit, name=image_names, method=method_names, sigma=noise_level,range=range_slide,scale=scale)
display(HBox(out.children[:-3]))#Show all controls
display(HBox(out.children[-3:-1]))
display(out.children[-1])#Show the output

Output()

In [142]:
name='Kurucz'
method='Wiener'
sigma=5
frame_path = os.path.join('Results','Results_spectra',name,name+'_'+'{:03d}.csv'.format(sigma))
df = pd.read_csv(frame_path,delimiter=',')
df[method]

0     -2.575262
1     -3.020788
2     -2.465971
3     -2.281697
4     -2.753901
         ...   
965    0.635679
966    0.675573
967    1.694407
968    1.367509
969    2.511206
Name: Wiener, Length: 970, dtype: float64